In [0]:
!pip install discord
!pip install pyspark

In [0]:

import discord
from discord.ext import commands
import pandas as pd
from datetime import datetime, timedelta
import pytz
import pyspark
from pyspark.sql.functions import col, from_utc_timestamp
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

In [0]:
# spark = SparkSession.builder.appName(
#   "pandas to spark").getOrCreate()

# spark = SparkSession.builder.appName('sparkdf').getOrCreate() 

In [0]:
import discord
import pandas as pd
from datetime import datetime, timedelta

def get_api_key(filename):
    with open(filename, 'r') as file:
        file_contents = file.read()
    return file_contents

API_TOKEN = get_api_key('overkey.txt')

intents = discord.Intents.default()
intents.message_content = True
intents.presences = True

client = discord.Client(intents=intents)

CHANNELS = ['lets-play-mm', 'lets-play-bmod','server-seeding']

intents = discord.Intents.default()
intents.message_content = True

client = discord.Client(intents=intents)

data = {'Author': [], 'Message': [], 'Timestamp': [], 'Channel': []}

async def collect_data(channel, data, msg_amount):
    async for message in channel.history(limit=msg_amount):
        author = message.author.name
        content = message.content
        timestamp = message.created_at
        msg_channel = message.channel

        # Append the data to the dictionary
        data['Author'].append(author)
        data['Message'].append(content)
        
        data['Timestamp'].append(timestamp)
        data['Channel'].append(msg_channel)

@client.event
async def on_ready():
    print(f'Logged in as {client.user.name} ({client.user.id})')
    print('------')
    df_list = []
    # Get the channel by name
    for channel_name in CHANNELS:
        channel = discord.utils.get(client.get_all_channels(), name=channel_name)
        data = {'Author': [], 'Message': [], 'Timestamp': [], 'Channel': []}
        # Run the data collection function
        await collect_data(channel, data, 200)
        
        # Create a DataFrame from the collected data
        df = pd.DataFrame(data)
        df = df.astype(str)
        substring = 'connect/'
        filter_condition = df['Message'].str.contains(substring, case=False, na=False)
        filtered_df = df[filter_condition]
        # print(df["Message"])
        # df['Channel'] = df['Channel'].replace('-', '_', regex=True)
        # df_list.append(df)    
        channel_name_replaced = channel_name.replace('-','_')

        spark_df = spark.createDataFrame(filtered_df)
        # spark_df.show()
        # spark_df.printSchema()
        # spark_df.saveAsTable("adb_hibak.bronze_discord_"+channel_name_replaced)
        spark.sql(f"DROP TABLE IF EXISTS adb_hibak.bronze_tf2_discord_{channel_name_replaced}")
        spark_df.write.mode('overwrite').format("delta").saveAsTable("adb_hibak.bronze_tf2_discord_"+channel_name_replaced)
    # for dfs in df_list:
    #     spark_df_connect = spark.createDataFrame(dfs)
    #     spark_df_connect.display()


        # df.to_csv(f'bronze\{channel_name}.csv')
        # df_dict[channel_name] = data
        # df_spark = spark.createDataFrame(data)
 
        # df_spark.show()
    

    await client.close()
    # print(df_dict)



await client.start(API_TOKEN)